Real quick recap, assembly language is a 1-1 translation of binary and high level code. The instructions you send to the CPU all have an OPCODE or a set of bits specifiying what the instruciton is, such as ADD. These instructions are followed by arguments or the things you supply after the instruction and the instructions fall into 4 general categories; arithmetic, logical, data transfer, and (un)conditional instructions. It seems like the instructions which are immediate (like addi) take one of their arguments from a register inside the CPU; makes sense since it is almost immediate or in the immediate vicinity. There is also no subi since addi can add a register and any number be it positive or negative. Remember, the default bias values is **127**. 

**Bitwise operations** are fast and simple instructions that operate on binary number's individual bits. Remember that since most of the bits in our numbers won't be used, sign extension (replacing unused bits with sign bit) will be used. So, yes, immediate numbers or arguments are not loaded from main memory but are used immediately in the program (a constant value or constant expression; one that you define in the instruction). **Directives** are kind of like tags or designations that tell the CPU which area of the program is used for storing data values like text, and which section is meant for storing the instructios in the program i.e. the assembly. You can think of it like the main function of a C(++), Java, or Go program. Directives are defined with a dot (.) prefix. Subsequent items are bound to that directive e.g. .asciiz "*str*" will store an array of 8 bit ASCII characters that are null terminated strings; look at the ISA for more info. **Labels** are text we use in an assembly language program to refer to a memory address with i.e. labels are text aliases for memory addresses; instead of typing in the address or register, you just use the label. Labels are used inside of directives and are proceeded by a colon (*label*<font color=red>:</font>). Labels' purpose is to mark lines of assembly, mark data/values stored in main memory, and so that you don't have to keep typing unsigned integers. 

In [ ]:
.data
age: .word 30
gpa: .half 0x10
hello: .asciiz "Hello, World!\n"

When it comes to storing data, you can use either main memory or registers. Registers, as we know, are pieces of hardware used to store values/data *inside* the CPU. In MIPS, all registers are preceeded with a $ and some are designated for special tasks such as the $v0 which is meant for holding the OPCODEs for syscalls while $a0 holds the arguments to those syscalls. Values for instructions must come from within the registers unless the instruction takes in a different arguments like a constant. While instructions like *add* and *sub* always use a register, immediate operands like *addi* don't use registers for getting values but simply storing results. Regsiters have both numbers (in the case of MIPS 0-31) *and* names such as the one I mentioned earlier with $v0.

<h2>Data Transfer Instructions</h2>

We have 2 types of instruction; instructions that load data from memory into a register and then instructions that store data from a register into memory. You can work with 4 sizes of data in MIPS, bytes (good for characters), half (16 bits or half a word), word, and doubles. These instructions either start with **l** (for load) or **s** (for store). The character proceeding that letter will specify the amount of data you want to load or store i.e. lw means you want to load a word of data from memory into a register. The arguemnt that follows in the register and is the register you either copy to or from, and the argument that follows tells you where in memory the data is or should be placed. 

In [ ]:
lw $t0, 16($sp)

The 16() serves as an offset telling the program that it should load a word of memory 16 bytes up from the stack pointer ($sp) into the register $t0. Pointer arithmetic is going to be (probably) prevalent in this course which of course gives us the power to screw over our program and kill the process it's running in! Quick side note: you MUST end your program with a syscall and $v0, 10; otherwise the program will continue to run! Sample program here

In [ ]:
main:
    li $t1, 1
    add $t0, $t1, 2
    li $v0, 10
    syscall

In [ ]:
# A demonstration of some simple MIPS instructions

	# All memory structures are placed after the
	# .data assembler directive
	.data

	# The .word assembler directive reserves space
	# in memory for a single 4-byte word (or multiple 4-byte words)
	# and assigns that memory location an initial value
	# (or a comma separated list of initial values)
value:	.word 12
Z:	.word 0

	# Declare main as a global function
	.globl main 

	# All program code is placed after the
	# .text assembler directive
	.text 		

# The label 'main' represents the starting point
main:
	li $t2, 25		# Load immediate value (25) 
	lw $t3, value		# Load the word stored in value (see the .data section at the top)
	add $t4, $t2, $t3	# Add
	sub $t5, $t2, $t3	# Subtract
	sw $t5, Z		# Store the answer in Z which is declared in the .data section at the top
    li $v0, 10              # syscall code 10 is for exit. # put it into $t0.
    syscall                 # make the syscall.

**Latches** are circuits that can store a bit of electricity allowing for memory. Latches can be gated with some extra logic gates to prevent reading and writing unless you have permission to do so, however latches by themselves aren't that great since they can only store a single bit. But if you put latches side by side, you can store larger pieces of data such as bytes, and if you follow this pattern you'll see that your registers in CPU will be made up of the architecture bit-length amount of latches. A problem comes when you want to individually access each latch for data however. The more latches you have, the more wires you'll need to access each one. To save on this hardware oversight, we can overlay wires over each other to make a grid which can be used to access each latch without a crap ton of wires. This approach requires some more logic gates but its all been done for us! Values used to access memory are split in half and fed into two different multiplexers which are used to select the row and column of memory that you're trying to access. IT'S ALL ABSTRACTION!!! So far, the registers we've looked at are general purpose, but there are a few more. *Special registers* can be used for multiplication and divsion, *floating point* registers are just what they sound like and are used for holding floating point values. General purpose registers just hold integers (which can represent a slew of things from intrusctions to characters, to just plain ol' numbers). **Register Files** are like groups of certain types of registers. The MIPS I has a register file for general purpose registers ($0-31) and a register file for floating point registers ($f0-31). There are also *counters* which are registers meant for starting with an intial value and then incrementing that value, like how a counter would. The **Program Counter** (PC) is another type of counter except one that is meant for keeping the address of the current instruction. Remeber how there was that instruction register that started at 0 and incremented too? The PC increments with each execution being made and so does the **Instruction Register** (IR) which keeps track of the instruction's location in memory. 

<h2>Instruction Formats</h2>

Instructions are formatted in a specific way e.g. *add $t0, $s0, $s1* turns into some binary for each operand and arguemnt. Although it's not layed out exactly as the instruction we see there are four parts to the instruction each of which is represented with its own byte. The first and last byte in an instruction is used to tell the CPU what operation is being performed. In MIPS the intruction will be 32 bits or 4 bytes. Also, since writing binary would not be very fun, you can use hex instead since its a power of 2. Each nybble (4 bits) can be represented as a single hex value. A segment of instruction is called a **field**. The first field is the OPCODE and tells you which operation is being performed while the last field is the *variant* of the OPCODE; you can look up what the other fields are, I feel like those are the most important. Usually we want each field to be the same width in terms of bits but if we're adding numbers, eventually we'll want to add numbers larger than a 32 bit number can represent. This first type of instruction format where all the fields are kept the same length is *R-type* or *R-format* standing for *register* type instructions. There is a second type of instruction format called *I-type* or *I-format* which is meant for immediate and data transfer instructions. The fields for an immediate instruction differ from register instruction with the first field being the OPCODE, the second the first register source, the second the second register source, and the third being a constant or address. Since the amount of fields differ for I-type and R-type instructions, the OPCODEs for each are encoded differently so that the hardware knows how to treat the instruction i.e. with equal sized fields or one wider field. 

To boil it down, **R-type commands only use values that are stored or should be stored in registers. I-type commands use *constants or addresses***. *op* = mnemonic for instruction / opcode, *funct* = variant of opcode, *rs* = source register, *rt* = second source register, *rd* = destination register. **OPCODE fields are 6 bits long but are always set to 0 in *R-format* with funct specifying the operation**. *shamt* = shift amount or the amount you shift up from the source (does this work with registers?). In I-type instructions *IMM* = immediate value and there is now shamt. IMM is used as the offset value (since you start at 0 with your addersses so you have to offset to their actual location). Ah, jumps – like the way they sound – *jump* to a label in the program based on conditions. Just to note, there are exceptions to the I and R formats and you can look them up. 

<h2>Addressing Modes</h2>

Since there are different formats opcodes and operands may differ, namely, the operands can be either registers or immediate values. These are also known as *modes*, or more formally, **address modes** for which there are five. These address modes are ways of getting data into the instruction by – you guessed – an address! Register addressing will take the address of a register and get a value from there. Jump addressing will get your value from somewhere you jump to, branch addressing will get you the address of a value based on the branch (this is all postulation but it seems correct!). And just to mention, there is also immediate addressing and base/displacement addressing (like setting the offset on a base address like 0 or like this offset(base) = 10($s3) offset is in bytes). Just like with binary arithmetic and its msb and lsb, instructions are formatted (instrucitons are the width of one word) in a way that have a most significant *byte* and a least significant byte. **Endianness** is what this arrangment thing is called. There are also endian types, where big endian means the msB is the most left one and the lsB is rightmost one, while in little endian, where this pattern is reversed. For this course, we use big endian! Because of the different ways that we can arrange our memory, we get the alignment problem (not the one with AI). By the way, your addresses are byte specific, but the addresses themselves are stored in words! That means you either have to multiply or divide by 4 to get to the right place in memory! So addresses that end in 4, 8, 12 (C) are validly aligned. 

Arrays are implemented in the way we've been told, with contiguous pieces of memory, each "element" being one of those contiguous pieces of memory. Remeber how labels could be used to identify certain lines in our program and as text aliases for locations in memory? – well, we can make arrays using that second fact. You can directly initialize the array or just allocate the memory, or set all the data in the allocated memory to nothing using the .space directive (this approach specifies the number of **bytes** to be allocated, this means there is a different in how many varaibles can be stored in the array and how much memory the array has i.e. integers in MIPS are 32 bits or 4 bytes so you can't assign an array with only 3 bytes of memory an integer value). Also remember that the label is an alias for the *first element* in memory and you have to index or offset to get to the others.

In [ ]:
.data
    scores: .word 100, 78, 43, 69, 21, 15, 88
    scores: .word 0:100
    scores: .space 400

.text
    la $rs, addr # 'la' stands for load address and is a psuedoinstruction that loads the actual address valye into a register

A way to get the nth index of an array is to take the first element i.e. the location in memory and load the address into a register, then take n and multiply it by 4 so that you're working in words (although I think this depends on the data type you're using) and add that number to the base address. Then you can load or store the value at that location. Here's what accessing an array might look like.

In [ ]:
.data

fibs: .word 0, 1, 1, 2, 3, 5, 8, 13, 21, 35, 55, 89, 144  # an array of integers --
                                                          # each integer is a word in length, 
                                                          # so each array element is 4 bytes away 
                                                          # from each other

.text

li $t1, 0         # put 0 into register $t1 to indicate 
                  # the element number of fibs[0] 
                  # that is, the very first element in the fibs array
                  
lw $t0, fibs($t1) # using base addressing mode with the label fibs 
                  # get the value at fibs[0] 
                  # and load that value into register $t0
                  
addi $t1, $t1, 4  # increase $t1 by 4 to represent $t1 indicating
                  # the element number of fibs[1]
                  # that is, the second element in the fibs array
                  
lw $t0, fibs($t1) # using base addressing mode with the label fibs 
                  # get the value at fibs[1] 
                  # and load that value into register $t0